In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy import optimize
import numdifftools

# Data

A data "Mode" from R package "mlogit" is used.  
Choose a transportation from "car", "carpool", "bus", and "rail" based on "cost" and "time".

In [6]:
data_wide = pd.read_csv("../data/mode_wide.csv", index_col=0)
data_wide.head()

,choice,cost.car,cost.carpool,cost.bus,cost.rail,time.car,time.carpool,time.bus,time.rail
1,car,1.507010,2.335612,1.800512,2.358920,18.503200,26.338233,20.867794,30.033469
2,rail,6.056998,2.896919,2.237128,1.855451,31.311107,34.256956,67.181889,60.293126
3,car,5.794677,2.137454,2.576385,2.747479,22.547429,23.255171,63.309057,49.171643
4,car,1.869144,2.572427,1.903518,2.268276,26.090282,29.896023,19.752704,13.472675
5,car,2.498952,1.722010,2.686000,2.973866,4.699140,12.414084,43.092039,39.743252


In [7]:
data_long = pd.read_csv("../data/mode_long.csv", index_col=0)
data_long.head(10)

,choice,alt,cost,time,chid
1.bus,False,bus,1.800512,20.867794,1
1.car,True,car,1.507010,18.503200,1
1.carpool,False,carpool,2.335612,26.338233,1
1.rail,False,rail,2.358920,30.033469,1
2.bus,False,bus,2.237128,67.181889,2
2.car,False,car,6.056998,31.311107,2
2.carpool,False,carpool,2.896919,34.256956,2
2.rail,True,rail,1.855451,60.293126,2
3.bus,False,bus,2.576385,63.309057,3
3.car,True,car,5.794677,22.547429,3


# Multinomial Logit Model

In [8]:
class MNL:
    def __init__(self, data, factor, choice="choice", alt="alt"):
        self.data = data
        self.factor = factor
        self.choice = choice
        self.alt = alt
        
        self.n_alt = len(self.data[self.alt].unique())
        self.n_chid = int(len(self.data) / self.n_alt)
        self.n_factor = len(self.factor)
    
    def LL(self, beta):

        # params
        a = beta[0:self.n_alt-1]
        b = beta[self.n_alt-1:self.n_alt+self.n_factor]

        LL_ = 0
        for n in range(self.n_chid):
            term1 = 0
            term2 = 0
            for i in range(n * self.n_alt, (n+1) * self.n_alt):
                const = 0 if i%self.n_alt == 0 else a[i%self.n_alt-1]
                bx = sum(b[k]*self.data[x][i] for k, x in enumerate(self.factor)) + const
                term2 += np.exp(bx)
                if self.data[self.choice][i] is np.bool_(True):
                    term1 = bx
            LL_ += term1 - np.log(term2)

        return -LL_  
    
    def LL0(self, beta):

        # params
        a = beta[0:self.n_alt-1]
        b = beta[self.n_alt-1:self.n_alt+self.n_factor]

        LL_ = 0
        for n in range(self.n_chid):
            term1 = 0
            term2 = 0
            for i in range(n * self.n_alt, (n+1) * self.n_alt):
                const = 0 if i%self.n_alt == 0 else a[i%self.n_alt-1]
                bx = const
                term2 += np.exp(bx)
                if self.data[self.choice][i] is np.bool_(True):
                    term1 = bx
            LL_ += term1 - np.log(term2)

        return -LL_
    
    def coefficients(self):
        coeff = self.data[self.alt].unique()[1:self.n_alt].tolist() 
        for i in range(self.n_alt-1):
            coeff[i] = coeff[i]+":(intercept)"
        coeff = coeff + self.factor 
        return coeff
    
    def run(self, beta):
        # about data
        print("number of choice")
        print(self.data[self.data[self.choice] == True].groupby(self.alt).count()[self.choice])
        
        # estimate
        result = optimize.minimize(self.LL, beta, method="L-BFGS-B")
        result0 = optimize.minimize(self.LL0, beta, method="L-BFGS-B")

        beta_opt = result.x
        hess = numdifftools.core.Hessian(self.LL)(beta_opt)
        stdev = np.sqrt(np.diagonal(np.linalg.inv(hess)))

        LL = -result.fun
        LL0 = -result0.fun
        
        # evaluate
        n_beta = self.n_alt - 1 + self.n_factor
        print()
        print("Evaluation index")
        print("log-Likelihood:", LL)
        print("McFadden R2:", 1-(LL/LL0))
        print("Adjusted McFadden R2:", 1-((LL-n_beta)/LL0))    
    
        t = np.zeros(n_beta)
        p = np.zeros(n_beta)
        signif_codes = []
        N = len(data_wide)
        for i in range(n_beta):
            t[i] = beta_opt[i] / stdev[i]
            alpha = stats.t.cdf(abs(t[i]), df=N-1)
            p[i] = (1-alpha) * 2
            if p[i] < 0.001:
                signif_codes.append("***")
            elif p[i] < 0.01:
                signif_codes.append("**")
            elif p[i] < 0.05:
                signif_codes.append("*")
            elif p[i] < 0.1:
                signif_codes[i].append(".")
            else:
                signif_codes[i].append("")

        summary = pd.DataFrame({"Coefficients": mnl.coefficients(),
                     "Estimate": beta_opt,
                     "Std. Error": stdev,
                     "t-value": t,
                     "p-value": p,
                     "": signif_codes})
        
    def result():
        print()
        print("Estimated parameters")
        print(summary)
        print()
        print("Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1")

In [9]:
len(data_wide)

453

In [11]:
len(data_long.chid.unique())

453

In [34]:
mnl = MNL(data=data_long, factor=["cost", "time"])
beta0 = [0, 0, 0, 0, 0]
mnl.run(beta0)

number of choice
alt
bus         81
car        218
carpool     32
rail       122
Name: choice, dtype: int64

Evaluation index
log-Likelihood: -354.4533477185816
McFadden R2: 0.34811344509556685
Adjusted McFadden R2: 0.3389177842406156

Estimated parameters
          Coefficients  Estimate  Std. Error    t-value       p-value     
0      car:(intercept)  3.292461    0.317276  10.377261  0.000000e+00  ***
1  carpool:(intercept) -0.905161    0.245943  -3.680374  2.609928e-04  ***
2     rail:(intercept)  0.627766    0.163361   3.842813  1.390411e-04  ***
3                 cost -0.772347    0.091979  -8.396951  6.661338e-16  ***
4                 time -0.085357    0.007748 -11.016126  0.000000e+00  ***

Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
